In [1]:
# Fetch COPOM statements from BCB API
import pandas as pd
import html
import re
from bs4 import BeautifulSoup

def fetch_copom_comunicado(meeting_number):
    """Fetch and clean COPOM statement for a given meeting number."""
    def clean_html(raw_html):
        if not raw_html:
            return ""
        decoded = html.unescape(raw_html)
        decoded = re.sub(r'[\u200b\ufeff\u00a0]', '', decoded)
        soup = BeautifulSoup(decoded, 'html.parser')
        
        # Remove tables
        for table in soup.find_all('table'):
            table.decompose()
        
        for br in soup.find_all('br'):
            br.replace_with('\n')
        for p in soup.find_all('p'):
            p.insert_after('\n\n')
        
        text = soup.get_text()
        text = re.sub(r'\n\s*\n', '\n\n', text)
        text = re.sub(r'[ \t]+', ' ', text)
        return text.strip()
    
    url = f"https://www.bcb.gov.br/api/servico/sitebcb/copom/comunicados_detalhes?nro_reuniao={meeting_number}"
    df = pd.read_json(url)
    content = df.iloc[0].iloc[0]
    
    return {
        "meeting": content.get('nro_reuniao'),
        "date": content.get('dataReferencia'),
        "title": content.get('titulo'),
        "text_pt": clean_html(content.get('textoComunicado'))
    }

# Fetch recent COPOM statements (e.g., meetings 260-274)
meetings = range(260, 275)
data = []

print("Fetching COPOM statements...")
for m in meetings:
    try:
        stmt = fetch_copom_comunicado(m)
        data.append(stmt)
        print(f"  Meeting {m}: {stmt['date'][:10] if stmt['date'] else 'N/A'}")
    except Exception as e:
        print(f"  Meeting {m}: Error - {e}")

print(f"\nLoaded {len(data)} statements")
data[:2]  # Preview first two

Fetching COPOM statements...
  Meeting 260: 2024-01-31
  Meeting 261: 2024-03-20
  Meeting 262: 2024-05-08
  Meeting 263: 2024-06-19
  Meeting 264: 2024-07-31
  Meeting 265: 2024-09-18
  Meeting 266: 2024-11-06
  Meeting 267: 2024-12-11
  Meeting 268: 2025-01-29
  Meeting 269: 2025-03-19
  Meeting 270: 2025-05-07
  Meeting 271: 2025-06-18
  Meeting 272: 2025-07-30
  Meeting 273: 2025-09-17
  Meeting 274: 2025-11-05

Loaded 15 statements


[{'meeting': 260,
  'date': '2024-01-31',
  'title': 'Copom reduz a taxa Selic para 11,25% a.a.',
  'text_pt': 'O ambiente externo segue volátil, marcado pelo debate sobre o início da flexibilização de política monetária nas principais economias e por sinais de queda dos núcleos de inflação, que ainda permanecem em níveis elevados em diversos países. Os bancos centrais das principais economias permanecem determinados em promover a convergência das taxas de inflação para suas metas em um ambiente marcado por pressões nos mercados de trabalho. O Comitê avalia que o cenário segue exigindo cautela por parte de países emergentes.\n\nEm relação ao cenário doméstico, o conjunto dos indicadores de atividade econômica segue consistente com o cenário de desaceleração da economia antecipado pelo Copom. A inflação cheia ao consumidor, conforme esperado, manteve trajetória de desinflação, assim como as medidas de inflação subjacente, que se aproximam da meta para a inflação nas divulgações mais rec

In [2]:
# 0) data: list[{"meeting": 274, "date":"2024-03-20", "text_pt": "..."}]

import numpy as np
import spacy
from sentence_transformers import SentenceTransformer, util
from scipy.signal import find_peaks

nlp = spacy.load("pt_core_news_md")  # or en_core_web_md if needed
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")  # swap to FinBERT/SBERT-pt if available

def to_sentences(txt):
    return [s.text.strip() for s in nlp(txt).sents if s.text.strip()]

def rolling_divergence(emb, k_base=20, k_local=5):
    N = len(emb)
    div = np.zeros(N)
    for i in range(N):
        base_start = max(0, i - k_base)
        base_vecs = emb[base_start:i] if i>0 else emb[0:1]
        sim = util.cos_sim(emb[i], base_vecs).mean().item()
        div[i] = 1 - sim
    return div

def process_statement(text):
    sents = to_sentences(text)
    emb = model.encode(sents, normalize_embeddings=True)
    div = rolling_divergence(emb, k_base=20)
    peaks, _ = find_peaks(div, prominence=0.08)  # tune
    return sents, div, peaks


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [3]:
# Process first statement as example
sents, div, peaks = process_statement(data[0]["text_pt"])
print(f"Meeting {data[0]['meeting']} - {len(sents)} sentences")
print(f"Peak divergence indices: {peaks}")

Meeting 260 - 25 sentences
Peak divergence indices: [2 5]


In [ ]:
import matplotlib.pyplot as plt

# Process all statements and collect results
results = []
for stmt in data:
    sents, div, peaks = process_statement(stmt["text_pt"])
    results.append({
        "meeting": stmt["meeting"],
        "date": stmt["date"][:10],
        "n_sentences": len(sents),
        "sentences": sents,
        "divergence": div,
        "peaks": peaks,
        "mean_div": np.mean(div),
        "max_div": np.max(div)
    })

# 1) Plot divergence over sentences for a single meeting
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Pick a meeting to analyze in detail
idx = -1  # Latest meeting
r = results[idx]

# Divergence curve
ax1 = axes[0, 0]
ax1.plot(r["divergence"], 'b-', lw=1.5)
ax1.scatter(r["peaks"], r["divergence"][r["peaks"]], c='red', s=100, zorder=5, label='Peaks')
ax1.set_xlabel("Sentence Index")
ax1.set_ylabel("Divergence (1 - cosine sim)")
ax1.set_title(f"Meeting {r['meeting']} ({r['date']}) - Sentence Divergence")
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2) Mean divergence across meetings (policy shift indicator)
ax2 = axes[0, 1]
dates = [r["date"] for r in results]
mean_divs = [r["mean_div"] for r in results]
ax2.plot(dates, mean_divs, 'go-', lw=2, markersize=8)
ax2.set_xlabel("Meeting Date")
ax2.set_ylabel("Mean Divergence")
ax2.set_title("Average Semantic Divergence Over Time")
ax2.tick_params(axis='x', rotation=45)
ax2.grid(True, alpha=0.3)

# 3) Number of peaks per meeting (complexity/novelty indicator)
ax3 = axes[1, 0]
n_peaks = [len(r["peaks"]) for r in results]
ax3.bar(dates, n_peaks, color='coral', edgecolor='black')
ax3.set_xlabel("Meeting Date")
ax3.set_ylabel("Number of Peaks")
ax3.set_title("Semantic Shift Points per Statement")
ax3.tick_params(axis='x', rotation=45)

# 4) Max divergence (biggest single shift)
ax4 = axes[1, 1]
max_divs = [r["max_div"] for r in results]
ax4.plot(dates, max_divs, 'rs-', lw=2, markersize=8)
ax4.set_xlabel("Meeting Date")
ax4.set_ylabel("Max Divergence")
ax4.set_title("Maximum Semantic Shift per Statement")
ax4.tick_params(axis='x', rotation=45)
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Show high-divergence sentences (potential policy shifts)
print(f"\n=== Key Sentences at Divergence Peaks (Meeting {r['meeting']}) ===\n")
for p in r["peaks"][:5]:  # Top 5 peaks
    print(f"[Sentence {p}] (div={r['divergence'][p]:.3f})")
    print(f"  {r['sentences'][p][:200]}...")
    print()